In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.stats import mode
from matplotlib import pyplot as plt
import seaborn as sns
from palettable.cartocolors.sequential import SunsetDark_6_r

## Data Validation

In [2]:
# change font sizes and whatnot
plt.rc("text", usetex=False)
plt.rc("xtick", labelsize=16)
plt.rc("ytick", labelsize=16)
plt.rc("axes", labelsize=16)

In [3]:
chains = np.load("prior/benzonitrile/chain.npy")
# hurr = np.load("cnn/2-cyanonapthalene/third.npy")

# chains = np.concatenate([hurr, chains], axis=0)

chains.shape

# Take the last 1000 samples of each walker, and then flatten
# along the walker dimension; i.e. get samples for each parameter
chains = chains[-1000:,:,:].reshape(-1, 14)

# calculate the percentiles used throughout the paper
summary_stats = np.percentile(chains, [16, 50, 84], 0)

# 16th, 50th, and 84th percentile
summary_stats.shape

_ = np.random.seed(42)

In [4]:
# plt.plot(chains[-1000:,:,9])

In [5]:
summary_stats

array([[9.17343635e+01, 7.16103150e+01, 2.32701381e+02, 2.47599504e+02,
        2.06749407e+11, 7.03897522e+11, 3.65949224e+11, 5.54173563e+11,
        6.03300620e+00, 5.57339158e+00, 5.76607442e+00, 5.89142269e+00,
        6.01660298e+00, 1.25753442e-01],
       [9.17797151e+01, 7.16602667e+01, 2.32743372e+02, 2.47641895e+02,
        2.24774982e+11, 7.25940228e+11, 3.87305019e+11, 5.72754822e+11,
        6.03343813e+00, 5.57561103e+00, 5.76776109e+00, 5.89348117e+00,
        6.01828099e+00, 1.26034786e-01],
       [9.18240687e+01, 7.17064749e+01, 2.32785068e+02, 2.47682774e+02,
        2.42037608e+11, 7.48670435e+11, 4.07802534e+11, 5.91309484e+11,
        6.03387016e+00, 5.57776428e+00, 5.76946432e+00, 5.89555359e+00,
        6.02001992e+00, 1.26307246e-01]])

In [6]:
# column names for each parameter
param_names = [
    r'Source Size #1 ["]',
    r'Source Size #2 ["]',
    r'Source Size #3 ["]',
    r'Source Size #4 ["]',
    r'N$_\mathrm{col}$ #1 [$10^{11}$ cm$^{-2}$]',
    r'N$_\mathrm{col}$ #2 [$10^{11}$ cm$^{-2}$]',
    r'N$_\mathrm{col}$ #3 [$10^{11}$ cm$^{-2}$]',
    r'N$_\mathrm{col}$ #4 [$10^{11}$ cm$^{-2}$]',
    r'T$_\mathrm{ex}$ [K]',
    r'V$_\mathrm{LSRK}$ #1 [km s$^{-1}$]',
    r'V$_\mathrm{LSRK}$ #2 [km s$^{-1}$]',
    r'V$_\mathrm{LSRK}$ #3 [km s$^{-1}$]',
    r'V$_\mathrm{LSRK}$ #4 [km s$^{-1}$]',
    r'dV [km s$^{-1}$]'
]

In [7]:
df = pd.DataFrame(
    chains,
    columns=param_names
)

# hacker statistics (not really)
# we don't need to use the full 20k samples to make plots, just take 1000 samples
# from the "posterior"
samples = df.sample(1000, replace=True)

# For the column densities, bring the numbers down to help make the figure formatting
# nicer looking
for i in range(4, 8):
    samples[param_names[i]] /= 1e11

In [8]:
top_color, bottom_color = SunsetDark_6_r.hex_colors[0], SunsetDark_6_r.hex_colors[-1]

In [ ]:
grid = sns.PairGrid(samples, corner=True, diag_sharey=False)
# grid.map_diag(sns.distplot, kde=False, color=top_color, norm_hist=True)# cumulative=False)
grid.map_diag(plt.hist, bins=15, color=top_color, linewidth=1.5, edgecolor="k", alpha=0.4, zorder=10)
grid.map_lower(sns.kdeplot, shade=True, shade_lowest=True, cmap=SunsetDark_6_r.get_mpl_colormap(), n_levels=6)
# loop over the diagonal of the corner plot, which are CDFs. We want to annotate
# the percentiles in this plot
for index, ax in enumerate(grid.diag_axes):
    name = param_names[index]
    param_stats = np.copy(summary_stats[:,index])
    if index in [4, 5, 6, 7]:
        param_stats /= 1e11
    # draw the region comprising 16th to 84th percentiles
    ax.axvspan(
        min(param_stats),
        max(param_stats),
        lw=1.5,
        color=bottom_color,
        facecolor="k",
        alpha=0.7,
        zorder=2
    )
    ranges = param_stats - param_stats.mean()
    ax.axvline(param_stats[1], lw=1.5, ls="--", color=top_color)
    # format the percentile stats in the title. This is somewhat of a pain because
    # we are combining LaTeX strings with Python f-strings
    lower_text = r"_\mathrm{" + f"{ranges[0]:.3f}" + r"}"
    upper_text = r"^\mathrm{" + f"+{ranges[-1]:.3f}" + r"}$"
    middle_text = f"{param_stats[1]:.3f}"
    ax.set_title(
        f"{name}\n" + r"$" + middle_text + lower_text + upper_text, fontsize=16
    )
# for i in range(1, 3):
#     grid.axes[i][0].set_ylim([0., 400.])
#     grid.axes[-1][i].set_xlim([0., 400.])
# for i in range(4, 8):
#     grid.axes[-1][i].set_xlim([1., 7.])
#     grid.axes[i][0].set_ylim([1., 7.])

grid.fig.tight_layout()

In [10]:
grid.fig.savefig("bn_prior_corner.pdf", dpi=300, transparent=True)